In [14]:
import pandas as pd 
import numpy as np
from numpy import array
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from ipynb.fs.full.Data_Processing import extract_test_number
from ipynb.fs.full.Data_Processing import get_samples
from sklearn import preprocessing
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
torch.set_printoptions(linewidth=120) #Display options for output
torch.set_grad_enabled(True) # Already on by default

In [5]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 8), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, 3)
        
    def forward(self, x):
        # Layer 1
        x = x.float()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)

        # FC Layer
        x = x.view(-1, 4*2*7)
        x = self.fc1(x)
        return x

**Evaluate function returns values of different criteria like accuracy, precision etc.**
In case you face memory overflow issues, use batch size to control how many samples get evaluated at one time. Use a batch_size that is a factor of length of samples. This ensures that you won't miss any samples.

In [6]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted , multi_class="ovr"))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted), average='macro'))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted) , average='macro'))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted) , average='macro')
            recall = recall_score(Y, np.round(predicted) , average='macro')
            results.append(2*precision*recall/ (precision+recall))
    return results

**Generate random data**
    
*Data format:*

Datatype - float32 (both X and Y)

X.shape - (#samples, 1, #timepoints, #channels)

Y.shape - (#samples)

In [7]:
X_train_ex = np.random.rand(100, 1, 120, 8).astype('float32') # np.random.rand generates between [0, 1)
y_train_ex = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.

X_val = np.random.rand(100, 1, 120, 8).astype('float32')
y_val = np.round(np.random.rand(100).astype('float32'))

X_test = np.random.rand(100, 1, 120, 8).astype('float32')
y_test = np.round(np.random.rand(100).astype('float32'))

### Loading the Data

### Load the data

In [21]:
inputs_and_labels = get_samples(sample_size=120)
inputs = inputs_and_labels['inputs']
targets_attention = inputs_and_labels['attention']

#Convert the categories into labels 
le = LabelEncoder()
targets_attention = le.fit_transform(targets_attention)

In [22]:
inputs.shape, targets_attention.shape

((6949, 120, 8), (6949,))

{0, 1, 2}

### Train Test Split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(inputs, targets_attention, test_size=0.20, random_state=42)

In [40]:
len(X_test)

1390

In [25]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

#Convert to 4D 
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0],1, X_test.shape[1],X_test.shape[2])





In [30]:
import torch.utils.data as data_utils

train = data_utils.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
test = data_utils.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)
test_loader = data_utils.DataLoader(test, batch_size=50, shuffle=False)

In [28]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [29]:
net = EEGNet()
optimizer = optim.Adam(net.parameters(), lr = 0.01)

preds_list = []
labels_list = []
for epoch in range(5):
    

    total_loss = 0
    total_correct = 0

    for batch in train_loader:
    #Get Batch
    
        inputs = batch[0]
        labels = batch[1]


        preds = net(inputs) #Pass batch
    #     criterion=nn.BCEWithLogitsLoss()
        loss = F.cross_entropy(preds, labels.long()) #calculate loss

        optimizer.zero_grad()
        loss.backward()  #calculate gradients
        optimizer.step() #update weights


        preds_list.append(preds)
        labels_list.append(labels)
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
    
    
#     Validation accuracy
#     params = ["acc", "fmeasure"]
#     print (params)
#     print ("Train - ", evaluate(net, X_train, y_train_new, params))
    print("epoch: {0}, Accuracy {1}, loss: {2}".format(epoch, total_correct/len(X_train), loss))

epoch: 0, Accuracy 0.9532289980212268, loss: 0.17539890110492706
epoch: 1, Accuracy 0.9870480302212629, loss: 0.0019378432771191
epoch: 2, Accuracy 0.9935240151106314, loss: 0.2706335783004761
epoch: 3, Accuracy 0.9823709300233855, loss: 0.008593092672526836
epoch: 4, Accuracy 0.9935240151106314, loss: 0.8923484086990356


In [45]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy of the network on {0} test samples: {1}".format(total, (correct/total * 100)))

Accuracy of the network on 1390 test samples: 98.92086330935251


In [98]:
# new training loo

for epoch in range(30):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    
    running_loss = 0.0
    for batch in train_loader:
        
        inputs = batch[0].to("cuda")
        labels = batch[1].to("cuda")

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data
    
    # Validation accuracy
    params = ["acc", "fmeasure"]
    print (params)
    print ("Training Loss ", running_loss)
    print ("Train - ", evaluate(net, X_train, y_train, params))
#     print ("Validation - ", evaluate(net, X_val, y_val, params))
#     print ("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0
['acc', 'fmeasure']
Training Loss  tensor(-21828.2227, device='cuda:0')
Train -  [0.0, nan]

Epoch  1


/cs/home/ybk1/python/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


['acc', 'fmeasure']
Training Loss  tensor(-21817.4297, device='cuda:0')
Train -  [0.0, nan]

Epoch  2
['acc', 'fmeasure']
Training Loss  tensor(-21808.3184, device='cuda:0')
Train -  [0.0, nan]

Epoch  3
['acc', 'fmeasure']
Training Loss  tensor(-21764.4453, device='cuda:0')
Train -  [0.0, nan]

Epoch  4
['acc', 'fmeasure']
Training Loss  tensor(-21782.6660, device='cuda:0')
Train -  [0.0, nan]

Epoch  5
['acc', 'fmeasure']
Training Loss  tensor(-21819.1113, device='cuda:0')
Train -  [0.0, nan]

Epoch  6
['acc', 'fmeasure']
Training Loss  tensor(-21773.5547, device='cuda:0')
Train -  [0.0, nan]

Epoch  7
['acc', 'fmeasure']
Training Loss  tensor(-21791.7773, device='cuda:0')
Train -  [0.0, nan]

Epoch  8
['acc', 'fmeasure']
Training Loss  tensor(-21837.3340, device='cuda:0')
Train -  [0.0, nan]

Epoch  9
['acc', 'fmeasure']
Training Loss  tensor(-21810., device='cuda:0')
Train -  [0.0, nan]

Epoch  10
['acc', 'fmeasure']
Training Loss  tensor(-21804.9570, device='cuda:0')
Train -  [0.0

**Run**

In [ ]:
batch_size = 32

for epoch in range(10):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(len(t10_AT_X_train)//batch_size-1): # x_train
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(t10_AT_X_train[s:e]) # x_train
        labels = torch.FloatTensor(np.array([t10_AT_y_train[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print (params)
    print ("Training Loss ", running_loss)
    print ("Train - ", evaluate(net, t10_AT_X_train, t10_AT_y_train, params))
#     print ("Validation - ", evaluate(net, X_val, y_val, params))
#     print ("Test - ", evaluate(net, X_test, y_test, params))